# A. Importing Library

In [1]:
from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
from sklearn.metrics import accuracy_score, precision_score, recall_score

import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# B. Defining functions

In [2]:
# nucleotide to unit vector
def one_hot(base):
  return np.array({'a':[1,0,0,0], 
                   't':[0,1,0,0], 
                   'g':[0,0,1,0], 
                   'c':[0,0,0,1]}[base.lower()])

# replace nucleotide for unit vector using one_hon()
def Hot(seq): 
  seq_one_hot = np.zeros((len(seq)*4, )) # zero 200,1 matrix 
  for i in range(len(seq)): seq_one_hot[4*i : 4*i+4] = one_hot(seq[i])
  return seq_one_hot

# normalize data (-1 to +1)
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# print results function
def print_results(results):
    print('Best Params: {}\n'.format(results.best_params_))
    
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means,stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean,3),round(std*2,3),params))

        
def clean_data(d1):
    # change column name
    d2 = d1.rename(columns={d1.columns[0]:'index',d1.columns[1]:'seq100',d1.columns[2]:'c26',d1.columns[3]:'c29', d1.columns[4]:'c31',d1.columns[5]:'c0'}) 


    # remove overhanger
    seq =[]
    for i in range(len(d2['seq100'])):
        seq.append(d2['seq100'][i][25:75])
    d3 = pd.DataFrame(seq,columns=['seq50'])
    d4 = pd.concat([d2,d3],axis=1)
    df = d4.drop(['index','seq100'],axis=1)

    #take a random sample of 10000
    #df = df.sample(n=len(df))
    df = df.sample(5000)
    return  df

def encode_dna_seq(df):
    # extract column of sequence
    SeqList = df['seq50'].tolist()

    # extract c0 values
    dy = np.around(df['c0'].values,3) 

    # encode dna sequence
    d = [] 
    for i in range(len(SeqList)): 
        d.append(Hot(SeqList[i])) 
    d = np.array(d)
    dx = pd.DataFrame(d)
    input = dx.values
    return input,dy

def split_data_tvt(dx,dy):
    x_train, x_test, y_train, y_test  = train_test_split(dx,dy,shuffle=True, test_size=0.40, random_state=1234)
    x_val, x_test, y_val, y_test  = train_test_split(x_test,y_test,shuffle=True, test_size=0.50, random_state=1234)
    return x_train, y_train, x_val, y_val, x_test, y_test

# C. Importing data

In [3]:
d = pd.read_csv('C:/Users/bciez/Documents/Basilio/jhu/ha-lab/dna_flexibility/thesis_01_18_2022/data/dataset1.txt',delimiter='\t')
print('ready')

ready


# D. Cleaning data

In [4]:
df = clean_data(d)
df.head(3)

,c26,c29,c31,c0,Amplitude,Phase,seq50
9262,0.424531,-0.089857,-0.777968,-0.081096,0.519038,-11.223393,CTCAAAACGCACAAAAATAAACATATGTATATATAGACATACACAC...
1761,-0.224606,-0.939009,-0.986435,-0.519606,-0.407753,-0.808827,TTCCAATTTAGCAACAATTGGTCAAAAGTATACTGACTCAAATCTT...
1568,0.855314,-1.093159,-0.412966,0.413702,-1.244731,-0.362684,CCATCACTGGTCCAAAAAAAGTACTTACCTACCCCACAAGATACAA...


# E. Encoding DNA sequence

In [5]:
dx,dy = encode_dna_seq(df)
print('ready')

ready


# F. Split data in training and testing set

In [6]:
x_train, y_train, x_val, y_val, x_test, y_test = split_data_tvt(dx,dy)
print('ready')

ready


# G. Gradient Boosting

In [7]:
gb = GradientBoostingRegressor()
parameters = {
    'n_estimators': [500,1000],
    'max_depth': [3,4],
    'learning_rate': [0.1]
}

cv = GridSearchCV(gb, parameters, cv = 5)
cv.fit(x_train, y_train)

print_results(cv)

Best Params: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500}

0.029 (+/-0.056) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
0.015 (+/-0.057) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000}
0.031 (+/-0.058) for {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500}
0.025 (+/-0.058) for {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 1000}


# H. Saving model

In [9]:
joblib.dump(cv.best_estimator_,'gb_model.pkl')

['gb_model.pkl']

# I. Best Model

In [10]:
cv.best_estimator_

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)